# Content Change Detection

Detects content changes using chunk-based comparison with version tracking.

## Overview

1. Load modified content from `content_repo` (ingested via [3_content_repo_edit.ipynb](3_content_repo_edit.ipynb))
2. **Chunk modified content and ingest into `content_chunks`** (happens in this notebook)
3. Load baseline chunks from `content_chunks` (v1 files)
4. Load current chunks from `content_chunks` (recently modified files)
5. Detect changes (NEW/MODIFIED/DELETED/UNCHANGED) by comparing checksums
6. Store results in `content_change_log`

## Architecture

- **Table**: `content_chunks` (NOT `content_checksums`)
- **Class**: `ChunkIngestion` (NOT `ChecksumIngestion`)
- **Relationship**: Chunks are linked to `content_repo` via FK `ud_source_file_id`
- **Versioning**: Compare v1 baseline vs v2+ current using `raw_file_version_nbr`

## Scope

This notebook handles:
- **Chunking**: Breaking modified content into fixed-size chunks
- **Checksum Computation**: Computing SHA-256 checksums for each chunk
- **Storage**: Ingesting chunks into `content_chunks` with FK relationships
- **Change Detection**: Comparing baseline (v1) vs current (v2+) chunks
- **Result Storage**: Recording detected changes

## Configuration

Set `SINCE_DATE` below to filter content modified after a specific date.

In [ ]:
# import sqlite3

# # Connect to your database
# conn = sqlite3.connect(DB_PATH)

# # Delete all entries
# conn.execute("DELETE FROM content_change_log")
# conn.commit()

# # Verify
# cursor = conn.execute("SELECT COUNT(*) FROM content_change_log")
# count = cursor.fetchone()[0]
# print(f"Remaining records: {count}")

Remaining records: 0


---

## Step 1: Setup and Configuration

In [2]:
import sys
import sqlite3
import pandas as pd
from pathlib import Path
from datetime import datetime
import hashlib
import uuid
from typing import Dict, List, Optional

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == "granular_impact" else NOTEBOOK_DIR.parent.parent

# Add PROJECT_ROOT to path so we can import granular_impact package
sys.path.insert(0, str(PROJECT_ROOT))

DATABASE_DIR = PROJECT_ROOT / "databases"
DB_PATH = DATABASE_DIR / "my_database.db"

print("=" * 100)
print("🔍 CONTENT CHANGE DETECTION")
print("=" * 100)
print(f"📁 Project Root: {PROJECT_ROOT}")
print(f"📁 Database: {DB_PATH}")
print(f"📁 Database exists: {DB_PATH.exists()}")
print("=" * 100)

🔍 CONTENT CHANGE DETECTION
📁 Project Root: c:\Users\praka\projects\FAQ\FAQ_update
📁 Database: c:\Users\praka\projects\FAQ\FAQ_update\databases\my_database.db
📁 Database exists: True


### Import Detection Modules and Chunking

In [4]:
from granular_impact.detection.checksum_extractor import ChecksumExtractor
from granular_impact.detection.content_change_detector import ContentChangeDetector
from granular_impact.detection.database_queries import ChangeDetectionQueries
from granular_impact.database.models import ChangeType
from granular_impact.chunking import SimpleChunker
from granular_impact.data_ingestion import ChunkIngestion

print("✅ Detection modules imported")

✅ Detection modules imported


### Configure Detection Parameters

In [5]:
SINCE_DATE = "2025-10-27 16:12:10"
SIMILARITY_THRESHOLD = 0.8
CHUNK_SIZE = 1000  # Same as in notebook 1
DETECTION_RUN_ID = f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{uuid.uuid4().hex[:8]}"

print(f"\nConfiguration:")
print(f"  Since Date: {SINCE_DATE}")
print(f"  Chunk Size: {CHUNK_SIZE} characters")
print(f"  Similarity Threshold: {SIMILARITY_THRESHOLD}")
print(f"  Run ID: {DETECTION_RUN_ID}")
print(f"\n{'='*80}")


Configuration:
  Since Date: 2025-10-27 16:12:10
  Chunk Size: 1000 characters
  Similarity Threshold: 0.8
  Run ID: run_20251027_130204_10a26d19



---

## Step 2: Load Modified Content

In [6]:
# First, let's verify the data and datetime filtering
with sqlite3.connect(DB_PATH) as conn:
    # Check all records with their modification dates
    all_records = pd.read_sql_query("""
        SELECT ud_source_file_id, raw_file_nme, last_modified_dt, file_status
        FROM content_repo
        WHERE file_status = 'Active'
        ORDER BY last_modified_dt DESC
    """, conn)
    
    print(f"\nTotal Active records: {len(all_records)}")
    print(f"\nAll modification dates:")
    print(all_records[['ud_source_file_id', 'raw_file_nme', 'last_modified_dt']].to_string())
    
    # Now filter with SINCE_DATE
    modified_content_df = pd.read_sql_query("""
        SELECT *
        FROM content_repo
        WHERE last_modified_dt > ? AND file_status = 'Active'
        ORDER BY last_modified_dt DESC
    """, conn, params=[SINCE_DATE])

print(f"\n{'='*80}")
print(f"Filtered records (since {SINCE_DATE}): {len(modified_content_df)}")
print(f"{'='*80}")

if len(modified_content_df) > 0:
    print("\nFiltered data preview:")
    display(modified_content_df[['ud_source_file_id', 'raw_file_nme', 'raw_file_version_nbr', 'last_modified_dt']].head(10))
else:
    print("\n⚠️ No records found! This might be a datetime format issue.")
    print(f"   SINCE_DATE value: {SINCE_DATE}")
    print(f"   Latest record date: {all_records['last_modified_dt'].max() if len(all_records) > 0 else 'N/A'}")


Total Active records: 24

All modification dates:
    ud_source_file_id               raw_file_nme     last_modified_dt
0                  15      Employee_Handbook.pdf  2025-10-27 16:12:36
1                  16     IT_Security_Policy.pdf  2025-10-27 16:12:36
2                  17     Remote_Work_Policy.pdf  2025-10-27 16:12:36
3                  18          Travel_Policy.pdf  2025-10-27 16:12:36
4                  19         Parental_Leave.pdf  2025-10-27 16:12:36
5                  20       Workplace_Safety.pdf  2025-10-27 16:12:36
6                  21       Health_Insurance.pdf  2025-10-27 16:12:36
7                  22        Vacation_Policy.pdf  2025-10-27 16:12:36
8                  23           Email_Policy.pdf  2025-10-27 16:12:36
9                  24       Equipment_Return.pdf  2025-10-27 16:12:36
10                  1      Employee_Handbook.pdf  2025-10-27 16:12:05
11                  2     IT_Security_Policy.pdf  2025-10-27 16:12:05
12                  3     Remote_Work_P

ud_source_file_id            raw_file_nme  raw_file_version_nbr  \
0                 15   Employee_Handbook.pdf                     2   
1                 16  IT_Security_Policy.pdf                     2   
2                 17  Remote_Work_Policy.pdf                     2   
3                 18       Travel_Policy.pdf                     2   
4                 19      Parental_Leave.pdf                     2   
5                 20    Workplace_Safety.pdf                     1   
6                 21    Health_Insurance.pdf                     2   
7                 22     Vacation_Policy.pdf                     2   
8                 23        Email_Policy.pdf                     2   
9                 24    Equipment_Return.pdf                     1   

      last_modified_dt  
0  2025-10-27 16:12:36  
1  2025-10-27 16:12:36  
2  2025-10-27 16:12:36  
3  2025-10-27 16:12:36  
4  2025-10-27 16:12:36  
5  2025-10-27 16:12:36  
6  2025-10-27 16:12:36  
7  2025-10-27 16:12:36  
8  2025-10-27 16:12:36  
9  2025-10-27 16:12:36

---

## Step 3: Chunk Modified Content and Ingest to content_chunks

This step chunks modified content and stores it in the `content_chunks` table with FK links to `content_repo`:

1. Read markdown files for modified content
2. Chunk into fixed-size pieces (1000 chars)
3. Compute SHA-256 checksums for each chunk
4. **Insert into `content_chunks` table with `ud_source_file_id` FK** (NOT `content_checksums`)

This creates the "current state" chunks (v2) that will be compared against baseline chunks (v1) to detect changes.

In [7]:
print(f"\n{'='*80}")
print("  CHUNKING AND CHECKSUM COMPUTATION")
print(f"{'='*80}\n")

# Initialize chunker and chunk ingestion
chunker = SimpleChunker(chunk_size=CHUNK_SIZE)
chunk_ingestion = ChunkIngestion(str(DB_PATH))

# Process each modified file
files_processed = 0
total_chunks = 0

for idx, row in modified_content_df.iterrows():
    # The path in database is relative to PROJECT_ROOT
    relative_path = row['extracted_markdown_file_path']
    markdown_path = PROJECT_ROOT / relative_path
    
    if not markdown_path.exists():
        print(f"⚠️  WARNING: File not found: {markdown_path}")
        continue
    
    # Read content
    try:
        content = markdown_path.read_text(encoding='utf-8')
    except Exception as e:
        print(f"✗ ERROR reading {markdown_path}: {e}")
        continue
    
    # Chunk and compute checksums
    chunks_with_checksums = chunker.chunk_with_checksums(content)
    
    # Ingest chunks for this file with FK to content_repo
    result = chunk_ingestion.ingest_chunks_for_file(
        ud_source_file_id=row['ud_source_file_id'],
        chunks_with_checksums=chunks_with_checksums,
        clear_existing=True
    )
    
    if result['success']:
        files_processed += 1
        total_chunks += result['rows_inserted']
        print(f"  ✓ {row['raw_file_nme']:40s} (v{row['raw_file_version_nbr']}) → {result['rows_inserted']:3d} chunks")
    else:
        print(f"  ✗ {row['raw_file_nme']:40s} - Error: {result['message']}")

print(f"\n✓ Processed {files_processed} files")
print(f"✓ Ingested {total_chunks} chunks into content_chunks table")
print(f"\n✅ Modified content chunked and stored in database")


  CHUNKING AND CHECKSUM COMPUTATION

  ✓ Employee_Handbook.pdf                    (v2) →   1 chunks
  ✓ IT_Security_Policy.pdf                   (v2) →   1 chunks
  ✓ Remote_Work_Policy.pdf                   (v2) →   1 chunks
  ✓ Travel_Policy.pdf                        (v2) →   1 chunks
  ✓ Parental_Leave.pdf                       (v2) →   1 chunks
  ✓ Workplace_Safety.pdf                     (v1) →   1 chunks
  ✓ Health_Insurance.pdf                     (v2) →   1 chunks
  ✓ Vacation_Policy.pdf                      (v2) →   1 chunks
  ✓ Email_Policy.pdf                         (v2) →   1 chunks
  ✓ Equipment_Return.pdf                     (v1) →   1 chunks

✓ Processed 10 files
✓ Ingested 10 chunks into content_chunks table

✅ Modified content chunked and stored in database


---

## Step 4: Load Baseline and Current Chunks, Then Detect Changes

Load chunks from `content_chunks` table (with JOIN to `content_repo` for version filtering):
- **Baseline**: Chunks from version 1 files (`raw_file_version_nbr = 1`)
- **Current**: Chunks from recently modified files (filtered by `last_modified_dt`)

Then detect changes by comparing checksums and content.

In [8]:
# Load baseline chunks (version 1 files)
with sqlite3.connect(DB_PATH) as conn:
    baseline_df = pd.read_sql_query("""
        SELECT cc.content_checksum, cc.chunk_text, cr.raw_file_nme
        FROM content_chunks cc
        JOIN content_repo cr ON cc.ud_source_file_id = cr.ud_source_file_id
        WHERE cr.raw_file_version_nbr = 1
          AND cc.status = 'active'
          AND cr.file_status = 'Active'
    """, conn)

print(f"Loaded {len(baseline_df)} baseline chunks (v1)")

# Load current chunks (recently modified files)
with sqlite3.connect(DB_PATH) as conn:
    current_df = pd.read_sql_query("""
        SELECT cc.content_checksum, cc.chunk_text, cr.raw_file_nme
        FROM content_chunks cc
        JOIN content_repo cr ON cc.ud_source_file_id = cr.ud_source_file_id
        WHERE cr.last_modified_dt > ?
          AND cc.status = 'active'
          AND cr.file_status = 'Active'
    """, conn, params=[SINCE_DATE])

print(f"Loaded {len(current_df)} current chunks (modified since {SINCE_DATE})")

# Initialize detector
detector = ContentChangeDetector(similarity_threshold=SIMILARITY_THRESHOLD)

# Prepare data structures for detector
previous_data = {}
for _, row in baseline_df.iterrows():
    previous_data[row['content_checksum']] = {
        'content_text': row['chunk_text'],
        'file_name': row['raw_file_nme']
    }

current_data = {}
for _, row in current_df.iterrows():
    current_data[row['content_checksum']] = {
        'text': row['chunk_text'],
        'file_name': row['raw_file_nme']
    }

print(f"\nPrepared {len(previous_data)} baseline checksums")
print(f"Prepared {len(current_data)} current checksums")

# Detect changes per file
all_changes = []
for file_name in modified_content_df["raw_file_nme"].unique():
    curr = {k: v for k, v in current_data.items() if v["file_name"] == file_name}
    prev = {k: v for k, v in previous_data.items() if v.get("file_name") == file_name}
    
    print(f"\nProcessing {file_name}:")
    print(f"  Current chunks: {len(curr)}")
    print(f"  Previous chunks: {len(prev)}")
    
    changes = detector.detect_changes(file_name, curr, prev, DETECTION_RUN_ID)
    all_changes.extend(changes)
    print(f"  Changes detected: {len(changes)}")

print(f"\n{'='*80}")
print(f"Total changes detected: {len(all_changes)}")
print(f"{'='*80}")

2025-10-27 13:02:07.125 | INFO     | granular_impact.detection.content_change_detector:__init__:83 - ContentChangeDetector initialized: algorithm=sha256, similarity_threshold=0.8, compute_llm_diffs=True, similarity_weights={'jaccard': 0.4, 'difflib': 0.6}
2025-10-27 13:02:07.129 | INFO     | granular_impact.detection.content_change_detector:detect_changes:121 - 🔍 Detecting changes for: Employee_Handbook.pdf
2025-10-27 13:02:07.130 | INFO     | granular_impact.detection.content_change_detector:detect_changes:122 -    Detection run: run_20251027_130204_10a26d19
2025-10-27 13:02:07.131 | INFO     | granular_impact.detection.content_change_detector:detect_changes:133 - 📊 Checksum analysis for Employee_Handbook.pdf:
2025-10-27 13:02:07.131 | INFO     | granular_impact.detection.content_change_detector:detect_changes:134 -    Current:   1 checksums
2025-10-27 13:02:07.132 | INFO     | granular_impact.detection.content_change_detector:detect_changes:135 -    Previous:  1 checksums
2025-10-27 

Loaded 16 baseline chunks (v1)
Loaded 10 current chunks (modified since 2025-10-27 16:12:10)

Prepared 16 baseline checksums
Prepared 10 current checksums

Processing Employee_Handbook.pdf:
  Current chunks: 1
  Previous chunks: 1
  Changes detected: 1

Processing IT_Security_Policy.pdf:
  Current chunks: 1
  Previous chunks: 1
  Changes detected: 1

Processing Remote_Work_Policy.pdf:
  Current chunks: 1
  Previous chunks: 1
  Changes detected: 1

Processing Travel_Policy.pdf:
  Current chunks: 1
  Previous chunks: 1
  Changes detected: 2

Processing Parental_Leave.pdf:
  Current chunks: 1
  Previous chunks: 1
  Changes detected: 2

Processing Workplace_Safety.pdf:
  Current chunks: 1
  Previous chunks: 1
  Changes detected: 1

Processing Health_Insurance.pdf:
  Current chunks: 1
  Previous chunks: 1
  Changes detected: 1

Processing Vacation_Policy.pdf:
  Current chunks: 1
  Previous chunks: 1
  Changes detected: 2

Processing Email_Policy.pdf:
  Current chunks: 1
  Previous chunks: 1

---

## Step 5: Store Results

In [9]:
if len(all_changes) > 0:
    with sqlite3.connect(DB_PATH) as conn:
        inserted = ChangeDetectionQueries.store_change_detection_results(
            conn, all_changes, DETECTION_RUN_ID, use_spark=False
        )
    print(f"✅ Stored {inserted} records")
    
    # Verify
    with sqlite3.connect(DB_PATH) as conn:
        verify_df = pd.read_sql_query("""
            SELECT change_type, COUNT(*) as count
            FROM content_change_log WHERE detection_run_id = ?
            GROUP BY change_type
        """, conn, params=[DETECTION_RUN_ID])
    display(verify_df)

2025-10-27 13:02:08.265 | INFO     | granular_impact.detection.database_queries:_store_changes_sqlite:278 - 💾 Stored 14/14 changes to database


✅ Stored 14 records


change_type  count
0    deleted_content      4
1   modified_content      4
2        new_content      4
3  unchanged_content      2

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
    verify_df = pd.read_sql_query("""
        SELECT *
        FROM content_change_log WHERE detection_run_id = ?
    """, conn, params=[DETECTION_RUN_ID])
display(verify_df)

change_id                                   content_checksum  \
0          43  f3bbae03d26ce982636aaa5552ad07e8735c7dd28f36d3...   
1          44  618294d1ec9e3f03260d3e7015c33734c3db3b8ffa543a...   
2          45  b83c5c009d7ae9abaaf977909eada2be578234e052751c...   
3          46  7317c6901c9282b3c1adbab4f0b5937f75d431737d2c49...   
4          47  c0a9947d707ea77c52566453c69cc6aef7cb3f07bcb742...   
5          48  72cb4a8e3750676f31506c2a95bc333bfaef0f916f020b...   
6          49  ac67a9a305fdff6919c6eb9e03a7ba2ae05cc0f991c2bb...   
7          50  30635deca8c6424f51a689016694ba5e2a3a4cc4a13fec...   
8          51  f728f4272c384ce30152a0bcf38e83d2bd3869f5a5be92...   
9          52  c6ce8f6d9959602f5e6fc2e1e135b0e4aa3e7874478588...   
10         53  c25998122f381276d80104faa040957acc631e97ad0274...   
11         54  691b8506507e69ae90edc609bb29735aa050604d10e575...   
12         55  26b8fffa2bb44d9ed01d6878fa75f7efc7c3d6355478e3...   
13         56  320f9f24dbf9c03edfdba60c2c0e973eff5c0932dc8183...   

                                    previous_checksum               file_name  \
0   b54c22ec37e28c72f242c5c96ba13a0d582cf14ad4bba7...   Employee_Handbook.pdf   
1   21f1d25d4f2dc7ce53d72cc050c32738362e1e12536d0b...  IT_Security_Policy.pdf   
2   525a301d7750b939ac2adf0e9f3690770b81277fd5dcc8...  Remote_Work_Policy.pdf   
3                                                None       Travel_Policy.pdf   
4                                                None       Travel_Policy.pdf   
5                                                None      Parental_Leave.pdf   
6                                                None      Parental_Leave.pdf   
7   30635deca8c6424f51a689016694ba5e2a3a4cc4a13fec...    Workplace_Safety.pdf   
8   370cd156d5ae42b3bb93fe51096b05d8dc2aee7358aaee...    Health_Insurance.pdf   
9                                                None     Vacation_Policy.pdf   
10                                               None     Vacation_Policy.pdf   
11                                               None        Email_Policy.pdf   
12                                               None        Email_Policy.pdf   
13  320f9f24dbf9c03edfdba60c2c0e973eff5c0932dc8183...    Equipment_Return.pdf   

    requires_faq_regeneration        change_type  similarity_score  \
0                           1   modified_content          0.950515   
1                           1   modified_content          0.954422   
2                           1   modified_content          0.803735   
3                           1        new_content          0.000000   
4                           1    deleted_content          0.000000   
5                           1        new_content          0.000000   
6                           1    deleted_content          0.000000   
7                           0  unchanged_content          1.000000   
8                           1   modified_content          0.957143   
9                           1        new_content          0.000000   
10                          1    deleted_content          0.000000   
11                          1        new_content          0.000000   
12                          1    deleted_content          0.000000   
13                          0  unchanged_content          1.000000   

   similarity_method                                          diff_data  \
0             hybrid  {"total_changes": 2, "change_types": {"replace...   
1             hybrid  {"total_changes": 2, "change_types": {"replace...   
2             hybrid  {"total_changes": 2, "change_types": {"replace...   
3             hybrid                                               None   
4             hybrid                                               None   
5             hybrid                                               None   
6             hybrid                                               None   
7             hybrid                                               None   
8             hybrid  {"total_changes": 1

: 

---

## Summary

In [ ]:
new = [c for c in all_changes if c.change_type == ChangeType.NEW_CONTENT]
modified = [c for c in all_changes if c.change_type == ChangeType.MODIFIED_CONTENT]
deleted = [c for c in all_changes if c.change_type == ChangeType.DELETED_CONTENT]

print("="*80)
print("✅ DETECTION COMPLETE")
print("="*80)
print(f"Run ID: {DETECTION_RUN_ID}")
print(f"Changes: {len(all_changes)} (NEW: {len(new)}, MOD: {len(modified)}, DEL: {len(deleted)})")
print("="*80)